In [2]:
from script import *

In [3]:
from bs4 import BeautifulSoup
import pandas as pd
import os
import traceback

**To Dos**
  - [ ] Account for staff.json files for atlant hawks starting on January 26th 2021

In [19]:
class BaseParser():
    ''''
    Base class for parsing out staff directories from html nba team pages
    '''
    def __init__(self,file):
        self.timestamp = file.split('/')[-1].split('.')[0]
        with open(file) as f:
            self.soup = BeautifulSoup(f)

        try:
            self.df = self.parse(self.soup)
        except Exception as e:
            traceback.print_exc()
            df = pd.DataFrame()
            df['timestamp'] = self.timestamp
            self.df = df
            pass

class BaseSuper(BaseParser):
    def __init__(self, *args, **kwargs):
        super(BaseSuper, self).__init__(*args, **kwargs)
        pass
    

class BostonParser(BaseSuper):
    def parse(self, soup):
        
        staff = []
        uls = soup.find("div", {"class": ["field-type-text-with-summary"]}).findChildren("ul" , recursive=True)
        for u in uls:
            dep = u.findPrevious('h2').get_text()
            dep_staff = [[f.strip() for f in li.get_text().replace('\n','').split('-',1)] + [dep] for li in u.find_all('li')]
            staff.append(dep_staff)

        staff = [t for s in staff for t in s]
        df = pd.DataFrame(staff, columns = ['name','position','department'])
        df['timestamp'] = self.timestamp
        return df



filename = 'data/raw/boston-celtics/20190411000957.html'
BostonParser(filename).df

,name,position,department,timestamp
0,Wyc Grousbeck,"Managing Partner, Governor & Chief Executive O...",Managing Partners,20190411000957
1,H. Irving Grousbeck,Managing Partner & Alternate Governor,Managing Partners,20190411000957
2,Stephen Pagliuca,Managing Partner & Alternate Governor,Managing Partners,20190411000957
3,Robert Epstein,The Abbey Group - Managing Partner & Alternate...,Managing Partners,20190411000957
4,Wendy Cooper,Executive Assistant,Managing Partners,20190411000957
...,...,...,...,...
143,Rick Coombs,Manager of Technology Programs and Services,Operations,20190411000957
144,Tim Knecht,Information Technology Administrator,Operations,20190411000957
145,Jason McKenna,Facilities Manager â€“ Practice Facility,Operations,20190411000957
146,James Parker,Associate Team Counsel,Operations,20190411000957


In [18]:
files = list(reversed(os.listdir('data/raw/boston-celtics')))
out = []
for f in files:
    out.append(BostonParser('data/raw/boston-celtics/{}'.format(f)).df)

[(o.shape,pd.unique(o.timestamp)) for o in out]


[((154, 4), array(['20210826214052'], dtype=object)),
 ((157, 4), array(['20210731102605'], dtype=object)),
 ((158, 4), array(['20210630080419'], dtype=object)),
 ((158, 4), array(['20210531011433'], dtype=object)),
 ((158, 4), array(['20210430074935'], dtype=object)),
 ((158, 4), array(['20210330160733'], dtype=object)),
 ((158, 4), array(['20210228055621'], dtype=object)),
 ((158, 4), array(['20210131083850'], dtype=object)),
 ((158, 4), array(['20201231073739'], dtype=object)),
 ((158, 4), array(['20201130204801'], dtype=object)),
 ((158, 4), array(['20201031225849'], dtype=object)),
 ((158, 4), array(['20200815102021'], dtype=object)),
 ((166, 4), array(['20191112050320'], dtype=object)),
 ((166, 4), array(['20190922122939'], dtype=object)),
 ((166, 4), array(['20190827010133'], dtype=object)),
 ((148, 4), array(['20190411000957'], dtype=object)),
 ((148, 4), array(['20181221012607'], dtype=object)),
 ((149, 4), array(['20181022090402'], dtype=object)),
 ((149, 4), array(['20180928

In [124]:
df = pd.concat(out)
df.drop_duplicates(['name','position']).to_csv('test.csv')
